In [6]:
import os, re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import urllib.request as ur
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


In [11]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36")
browser =  webdriver.Chrome('../chromedriver', options = options)

url = 'https://www.yogiyo.co.kr/mobile/#/234942/'
# headers = {'user-agent':Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}


# 링크 주소 가져오기
browser.get(url)
browser.maximize_window()

# 리뷰 버튼 클릭

time.sleep(3)

review_button = browser.find_elements_by_css_selector('.nav.nav-tabs.restaurant-tab a')
review_button[1].click()

# 더보기 클릭

# try :
#     more = browser.find_element_by_css_selector('li.list-group-item.btn-more a')
#     more.click()
# except :
#     pass

# 화면 가장 아래로 스크롤 내리기
browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

Interval = 3 # 2초에 한번씩


# 마지막 날짜(크롤링 시작 날짜) 20년 1월 1일~10일
first_days = []
for i in range(1,10,1):
    first_days.append("2021년 1월 {}일".format(i))
print(first_days)


# 현재 문서 높이를 가져와서 저장
prev_height = browser.execute_script("return document.body.scrollHeight")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    try:
        more = browser.find_element_by_css_selector('li.list-group-item.btn-more a')
        more.click()
    except:
        pass
    time.sleep(Interval)
    
    curr_height = browser.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break
    
    prev_height = curr_height
#     print(browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope span.review-time.ng-binding')[-1].text)
    
    # first_day에 정한 날짜까지 스크롤할 시 스크롤 중지
    if browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope span.review-time.ng-binding')[-1].text.strip() in first_days:
        print("{}까지 스크롤 완료".format(browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope span.review-time.ng-binding')[-1].text.strip()))
        break
        



# 별점 태그 ratings에 저장
ratings = browser.find_elements_by_css_selector('span.category span.points.ng-binding')


# 리뷰 reviews에 저장
reviews = browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope p')

# 카테고리 별 인덱싱
taste_idx = 0
quantity_idx = 1
delivery_idx = 2
review_idx = 0   

# 맛, 양, 배달 별 별점, 리뷰 리스트 선언
taste_ratings = []
quantity_ratings = []
delivery_ratings = []
overall_reviews = []



# 각 별점 중 하나라도 3점 이하면 review_bads에 append



while True:
    try:
        # 별점 중 하나라도 3점 이하면 각 별점 리스트 및 리뷰 리스트에 어펜드
        if int(ratings[taste_idx].text) <= 3 or int(ratings[quantity_idx].text) <= 3 or int(ratings[delivery_idx].text) <= 3:
            taste_ratings.append(ratings[taste_idx].text)
            quantity_ratings.append(ratings[quantity_idx].text)
            delivery_ratings.append(ratings[delivery_idx].text)
            overall_reviews.append(reviews[review_idx].text)
        else :
            pass
    except:
        break
    

    taste_idx += 3
    quantity_idx += 3
    delivery_idx += 3
    review_idx += 1


print('맛 평점: ', taste_ratings)
print('양 평점: ', quantity_ratings)
print('배달 평점: ', delivery_ratings)
print('전체', overall_reviews)





['2021년 1월 1일', '2021년 1월 2일', '2021년 1월 3일', '2021년 1월 4일', '2021년 1월 5일', '2021년 1월 6일', '2021년 1월 7일', '2021년 1월 8일', '2021년 1월 9일']
2021년 1월 4일까지 스크롤 완료
맛 평점:  ['1', '1', '3', '1', '4', '5', '1', '5', '5', '1', '4', '4', '5', '1', '5', '1', '5', '5', '3', '4', '4', '4', '4', '3', '5', '1', '5', '1', '3', '1', '1', '1', '2', '1', '5', '1', '2', '5', '1', '3', '3', '5', '3', '5', '5', '3', '3', '3', '3', '4', '1', '5', '1', '4', '1', '3', '4', '1', '2', '3', '1']
양 평점:  ['1', '1', '3', '1', '3', '3', '1', '3', '5', '1', '3', '4', '3', '2', '5', '2', '4', '5', '5', '4', '4', '3', '3', '5', '5', '1', '3', '1', '3', '1', '1', '5', '4', '1', '5', '1', '2', '5', '1', '4', '5', '5', '4', '3', '1', '3', '3', '3', '3', '5', '1', '5', '4', '5', '4', '3', '5', '1', '2', '3', '3']
배달 평점:  ['1', '1', '5', '1', '5', '5', '1', '5', '3', '1', '5', '1', '5', '5', '1', '1', '3', '3', '4', '2', '3', '3', '5', '3', '1', '1', '5', '1', '1', '1', '1', '2', '2', '1', '3', '1', '2', '1', '1', '4', '5', '2'

In [12]:
len(taste_ratings)

61

In [13]:
(overall_reviews)

['배달은 재 시간보다 40분 늦게 왔고... 감자튀김은 식어서 눅눅하고.. 햄버거 양상추는 시든것으로 오고.... 콜라가 제일 맛나더이다',
 '햄버거를 한시간 기다려서 먹으면 패스트푸드가 맞나요? 주문 30분 후 잘 받았냐, 맛있게 먹었냐는 왜 묻는지? 음식 출발은 45분만에 했다면서 롯데리아 배달 이따위로 할거면 요기요로 장사 집어치우세요',
 '양상추좀 많이 넣어달랬는데.. 본인 햄버거라면 이렇게 만드셨을지 참..',
 '오늘 진짜 맛없었습니다......양ㄴㅕㅁ도 거의없고 튀김은무슨 기름덩어리네요...진짜 욕나옵니다 누가 조리했는지 형편없네요',
 '감자튀김 양이 반절정도 빠져있던것도 있어서..... 다음에는 확실히 알차게 담아주세요..',
 '맛있고 다 좋은데 콜라 이만큼 줄거면 추가요금 받지 마시쥬,,? 사진이라 얼마 차이 안나보이지만 그냥 보통 사이즈랑 별 다를게 없던데요',
 '양념감자 2개, 닭가슴살 1개, 아이스 라떼 1개 주문했는데 1시간 걸렸어요. 양념감자 양 너무 적고 라떼는 아무리 롯데리아여도 거의 맹물 맛이네요. 너무 심해요.',
 '맛은 워낙 먹는거라 말도않고 배달 빨라좋았습니다',
 '콜라 포장부분에 바늘구멍같은 구멍이 나서 왔어요 음료 밖에 튀어나오고 괜히 찝찝하고 불안해서 그냥 버렸습니다',
 '아이스크림이 안왔네요?',
 '롯리는 신메뉴는 원래 한동안 좋아서 신메뉴기대했는데 이번엔 시작부터 별로네요. trex드세요',
 '진짜. 맨날 1시간 넘게걸림 ㅡㅡ',
 '맛있게 잘 먹었습니다...ㅎㅎ',
 '감자튀김이 하나빠져서왔오요 ㅡㅡ',
 '배달ㄹㅇ늦어염ㅜㅜㅜㅜㅜ',
 '오늘 비가 와서 배송이 늦긴 했는데 ..오다 혹시감자튀김 물에 빠뜨리신건지 ...너무 눅눅 하더라구요 ㅠㅠ',
 '배달이 늦어서 조금 식었지만 맛있었어요~!^^',
 '배달이 늦어서 다 식어서 왔어요ㅠㅠㅠ',
 '치즈넘버파이브 완전 차갑게 왔고, 버거도 맛없어요.',
 '사각새우더블버거를 시켰는데 오리지널 새우버거가 왔어요',
 '프로모션때문인지 배달이 좀 늦고

In [14]:
import konlpy
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Komoran

In [15]:
komoran = Komoran()

In [23]:
type(overall_reviews[0])

str

In [22]:
len(word_dict)

51

In [25]:
word_list=[]
for i in range(len(overall_reviews)):
    word_list.append(komoran.nouns(overall_reviews[i]))
    print(komoran.nouns(overall_reviews[i])) # 단어들을 각각 요소로 저장


['배달', '시간', '분', '감자', '튀김', '식', '햄버거', '상추', '것', '콜라', '제일', '맛나', '더']
['햄버거', '시간', '패스트푸드', '주문', '분', '후', '음식', '출발', '분만', '롯데리아', '배달', '따위', '할거', '요기', '요로', '장사']
['상추', '본인', '햄버거', '라면']
['오늘', '양념', '튀김', '은', '기름', '덩어리', '욕', '조리']
['감자', '튀김', '양이', '반절', '정도', '것', '다음']
['콜라', '줄', '거', '추가', '요금', '마시', '쥬', '사진', '얼마', '차이', '나보이', '지만', '사이즈']
['양념', '감자', '개', '닭', '가슴', '살', '개', '아이스', '라', '떼', '개', '주문', '시간', '양념', '감자', '라', '떼', '롯데리아', '맹물', '맛']
['맛', '거', '말', '배달']
['콜라', '포장', '부분', '바늘구멍', '구멍', '음료', '밖', '불안']
['아이스크림']
['롯', '리', '메뉴', '원래', '한동안', '좋아서', '메뉴', '기대', '이번', '시작']
['진짜', '시간']
[]
['감자', '튀김']
[]
['오늘', '비', '배', '송이', '오다', '감자', '튀김', '물', '건지']
['배달']
['배달', '식']
['치즈', '넘버', '파이브', '완전', '버거']
['사각', '새우', '더블', '버거', '오리지널', '새우', '버거']
['프로모션', '때문', '배달', '햄버거', '채', '배달']
['처음', '버거']
[]
['음식', '다식']
['새벽', '상품', '야채', '튀김', '배달', '라이', '더', '손', '음료', '손가락', '대행', '인가', '옷', '롯데리아', '옷', '배달', '친구', '거', '교육', '부탁', '손', 

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte

In [27]:
# word_list=[]
# for i in range(len(overall_reviews)):
#     temp_list = []
#     print(komoran.nouns(overall_reviews[i]))
#     for j in range(len(overall_reviews[i])):
#         temp_list.append(komoran.nouns(overall_reviews))
#         word_list.append(temp_list) # 단어들을 각각 요소로 저장


In [34]:
wordList = word_list
 
wordCount = {}
 
for word in wordList:

    # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

    wordCount[word] = wordCount.get(word, 0) + 1 
    
    keys = sorted(wordCount.keys())

for word in keys:

    print(word + ':' + str(wordCount[word])) 

NameError: name 'keys' is not defined

NameError: name 'keys' is not defined